In [45]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import geopy
from geopy.extra.rate_limiter import RateLimiter
from numpy import genfromtxt
from io import StringIO

In [46]:
# need to create the labels of the dataframe correctly
crime_df = pd.read_csv('CrimeReports.csv', sep=',', names=['Number', 'Offense Code', 'Offense Description',
                                                     'Date', 'Time', 'Street', 'City', 'State', 'Zip'])
print (crime_df.head(10)) #  First 5 rows


   Number Offense Code                              Offense Description  \
0       1  FRAUD-26A-1  FRAUD - FALSE PRETENSES/SWINDLE/CONFIDENCE GAME   
1       2  FRAUD-26C-1                            FRAUD - IMPERSONATION   
2       1   TRES-90J-3               TRESPASSING - PUBLIC/PARK PROPERTY   
3       1       SERV-8                    SERVICE - LOST/FOUND PROPERTY   
4       2     SERV-999                              POLICE PHONE REPORT   
5       1       SERV-1                         SERVICE - POLICE SERVICE   
6       1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
7       1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
8       2   DEST-290-2                   DESTRUCTION - PRIVATE PROPERTY   
9       1       SERV-1                         SERVICE - POLICE SERVICE   

         Date    Time                  Street          City State      Zip  
0  2021-04-24  1200.0      8200 ASPEN GLEN CT    ALEXANDRIA    VA  22309.0  
1  2021-04-24  1200.

In [47]:
# reduce the size of the dataframe to develop additional functions
crime_df = crime_df.head(25)
print (crime_df)

    Number Offense Code                              Offense Description  \
0        1  FRAUD-26A-1  FRAUD - FALSE PRETENSES/SWINDLE/CONFIDENCE GAME   
1        2  FRAUD-26C-1                            FRAUD - IMPERSONATION   
2        1   TRES-90J-3               TRESPASSING - PUBLIC/PARK PROPERTY   
3        1       SERV-8                    SERVICE - LOST/FOUND PROPERTY   
4        2     SERV-999                              POLICE PHONE REPORT   
5        1       SERV-1                         SERVICE - POLICE SERVICE   
6        1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
7        1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
8        2   DEST-290-2                   DESTRUCTION - PRIVATE PROPERTY   
9        1       SERV-1                         SERVICE - POLICE SERVICE   
10       1      SERV-29                       SERVICE - DOMESTIC DISPUTE   
11       1       SERV-1                         SERVICE - POLICE SERVICE   
12       1  

In [48]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="crime_data_locations")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

crime_df['Location'] = crime_df['Street'].apply(geocode)
crime_df['Point'] = crime_df['Location'].apply(lambda loc: tuple(loc.point) if loc else None)

print (crime_df) #  First 5 rows

    Number Offense Code                              Offense Description  \
0        1  FRAUD-26A-1  FRAUD - FALSE PRETENSES/SWINDLE/CONFIDENCE GAME   
1        2  FRAUD-26C-1                            FRAUD - IMPERSONATION   
2        1   TRES-90J-3               TRESPASSING - PUBLIC/PARK PROPERTY   
3        1       SERV-8                    SERVICE - LOST/FOUND PROPERTY   
4        2     SERV-999                              POLICE PHONE REPORT   
5        1       SERV-1                         SERVICE - POLICE SERVICE   
6        1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
7        1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
8        2   DEST-290-2                   DESTRUCTION - PRIVATE PROPERTY   
9        1       SERV-1                         SERVICE - POLICE SERVICE   
10       1      SERV-29                       SERVICE - DOMESTIC DISPUTE   
11       1       SERV-1                         SERVICE - POLICE SERVICE   
12       1  

In [49]:
crime_df[['Latitude', 'Longitude', 'Altitude']] = pd.DataFrame(crime_df['Point'].tolist(), index=crime_df.index)
#crime_df = pd.DataFrame(crime_df.Point.str.split(',',1).tolist(),
#                                 columns = ['Latitude', 'Longitude', 'Altitude'])
print (crime_df) #  First 5 rows

    Number Offense Code                              Offense Description  \
0        1  FRAUD-26A-1  FRAUD - FALSE PRETENSES/SWINDLE/CONFIDENCE GAME   
1        2  FRAUD-26C-1                            FRAUD - IMPERSONATION   
2        1   TRES-90J-3               TRESPASSING - PUBLIC/PARK PROPERTY   
3        1       SERV-8                    SERVICE - LOST/FOUND PROPERTY   
4        2     SERV-999                              POLICE PHONE REPORT   
5        1       SERV-1                         SERVICE - POLICE SERVICE   
6        1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
7        1   BURG-220-3                   BURGLARY/BREAKING AND ENTERING   
8        2   DEST-290-2                   DESTRUCTION - PRIVATE PROPERTY   
9        1       SERV-1                         SERVICE - POLICE SERVICE   
10       1      SERV-29                       SERVICE - DOMESTIC DISPUTE   
11       1       SERV-1                         SERVICE - POLICE SERVICE   
12       1  

In [50]:
"""def fix_nan_values(column):
    column_mean = crime_df[column].mean()
    crime_df = crime_df.fillna(column_mean)"""

In [57]:
crime_df['Point'].fillna((crime_df['Point'].mean()), inplace=True)

"""# check for NaN values in the Points Column
if (crime_df['Point'].isnull().values.any() == True):
    column_mean = crime_df['Point'].mean()
    crime_df = crime_df['Point'].fillna(column_mean)

#crime_df['Point'].isnull().sum()"""

TypeError: can only concatenate tuple (not "int") to tuple

In [54]:
map1 = folium.Map(
    location=[38.9085,-77.2405],
    tiles='OpenStreetMap',
    zoom_start=12,
)
crime_df.apply(lambda row:folium.Marker(location=[row['Latitude'], row['Longitude']]).add_to(map1), axis=1)
map1

ValueError: Location values cannot contain NaNs.